In [1]:
import pandas as pd 
import numpy as np 

In [31]:
# Number of rows (dates) and columns (random numbers)
num_rows = 10
num_fields = 5

columns = ['FGPR', 'FOPR', 'FWPR']
dfs = []
for i in range(num_fields):
    # Generate random numbers
    random_numbers = np.random.rand(num_rows, len(columns))
    # Create a date range for the index
    date_range = pd.date_range(start='2023-01-01', periods=num_rows, freq='D')

    # Create the DataFrame
    temp =pd.DataFrame(random_numbers, index=date_range, columns=columns)
    temp ['FIELD'] = f'FIELD_{i}'
    dfs.append(temp)

# Concatenate the DataFrames
df = pd.concat(dfs)
mother_field = {'FIELD_0':'FIELD', 'FIELD_1':'FIELD', 'FIELD_2':'FIELD_0', 'FIELD_3':'FIELD_1','FIELD_4':'FIELD_1'}
df['MOTHER_FIELD'] = df.FIELD.map(mother_field)
# Display the DataFrame

print(df)


                FGPR      FOPR      FWPR    FIELD MOTHER_FIELD
2023-01-01  0.569350  0.064821  0.402822  FIELD_0        FIELD
2023-01-02  0.711965  0.532466  0.774405  FIELD_0        FIELD
2023-01-03  0.556708  0.566616  0.117166  FIELD_0        FIELD
2023-01-04  0.418276  0.516058  0.249934  FIELD_0        FIELD
2023-01-05  0.703697  0.583807  0.913285  FIELD_0        FIELD
2023-01-06  0.424742  0.572168  0.030119  FIELD_0        FIELD
2023-01-07  0.670858  0.041518  0.523945  FIELD_0        FIELD
2023-01-08  0.800804  0.991528  0.009428  FIELD_0        FIELD
2023-01-09  0.549338  0.115837  0.767002  FIELD_0        FIELD
2023-01-10  0.006352  0.253510  0.334380  FIELD_0        FIELD
2023-01-01  0.502975  0.905707  0.639097  FIELD_1        FIELD
2023-01-02  0.004178  0.629150  0.481268  FIELD_1        FIELD
2023-01-03  0.496677  0.654601  0.131449  FIELD_1        FIELD
2023-01-04  0.416697  0.345486  0.008756  FIELD_1        FIELD
2023-01-05  0.580836  0.668856  0.530297  FIELD_1      

In [6]:
df.index.unique()

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10'],
              dtype='datetime64[ns]', freq=None)

In [52]:
export_file = 'eclipse_schedule.sch'
def make_schedule_function(df, export_file=export_file):
    eclipse_date ={1:'JAN',2:'FEB',3:'MAR',\
    4:'APR',5:'MAY',6:'JUN',7:'JUL',\
        8:'AUG',9:'SEP',10:'OCT',11:'NOV',12:'DEC'}

    with open(export_file, "w") as f:
        f.write("------------------------------------------\n")
        f.write("-- Automatic generation of the GRUPTREE --\n")
        f.write("GRUPTREE \n")
        for field in df['FIELD'].unique():
           f.write(f" '{field}'       '{df.loc[df.FIELD==field, 'MOTHER_FIELD'].unique()[0]}' / --fields to be tied in to FIELD \n")
        f.write("/ \n")

        f.write("------------------------------------------\n")
        f.write("-- Automatic generation of the GSATPROD --\n")
        f.write("-- Based on dummy exports-- \n")
        f.write("------------------------------------------------ \n")
        for selected_date in df.index.unique():
            f.write("DATES \n")
            f.write(f"{selected_date.day}  {eclipse_date[selected_date.month]}  {selected_date.year}  / \n")
            f.write("/ \n")
            f.write("GSATPROD \n")
            for field in df.FIELD.unique():
                if not (df.loc[(df.index == selected_date)&(df.FIELD == field)]).empty :
                    oil = df.loc[(df.index == selected_date) &(df.FIELD == field), 'FOPR'].values[0]
                    gas = df.loc[(df.index == selected_date) &(df.FIELD == field), 'FGPR'].values[0]
                    water = df.loc[(df.index == selected_date) &(df.FIELD == field), 'FWPR'].values[0]
                    f.write(f"'{field}'  {oil:.2f}  {gas:.2f}  {water:.2f}  3*  / \n")
                f.write("/ \n")
        print("file generated")




                        

In [53]:
make_schedule_function(df)

file generated


In [40]:
field ='FIELD_1'
df.loc[df.FIELD==field, 'MOTHER_FIELD'].unique()

array(['FIELD'], dtype=object)

In [26]:
df.loc[(df.index == df.index.unique()[0]) &(df.FIELD == field), 'FOPR'].values[0]

0.038326006463034745

In [49]:
df.loc[df.FIELD==field, 'MOTHER_FIELD'].unique()[0]

'FIELD'